In [116]:
import pandas as pd
import numpy as np

import requests
import bs4 as bs
from urllib.parse import urljoin
import pickle

import os

%matplotlib inline

# Test (Spain)

https://en.wikipedia.org/wiki/Category:Opinion_polling_in_Spain

In [37]:
#Get all election links

#restrictions in link
restrictions = ["Opinion polling","general election"]
#main url
url = "https://en.wikipedia.org/wiki/Category:Opinion_polling_in_Spain"

#Extract all links
links = []
r = requests.get(url)
if r.ok:
    html = bs.BeautifulSoup(r.text,"html.parser")
    for link in html.find_all("a"):
        if link.has_attr('href') and all([restr in link.text for restr in restrictions]): #check restrictions
            links.append(urljoin(url,link["href"]))
print(links)
                    
                    

['https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1982', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1986', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1989', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1993', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1996', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2000', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2004', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2008', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2011', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2015', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2016', 'https://en.wikipedia.org/wiki/

In [100]:
def check_not_number(x):
    try:
        float(x.split("/")[0])
        return False
    except:
        return True

In [149]:
all_tables = []
country = "Spain"
!mkdir data/individual_tables/$country

#position table in website
table_position = 0
for p_url in links:
    #Main ulr
    #p_url = "https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1982"

    #Get columns
    r = requests.get(p_url)
    html = bs.BeautifulSoup(r.text,"html.parser")
    tables = html.find_all("table")[table_position]

    rows = tables.find_all("th")
    columns = [r.find("img")["alt"] if (r.find("img") is not None) else (r.find("a")["href"] if (r.find("a") is not None) else None) for r in rows ]
    columns = [_ for _ in columns if _ is not None]
    
    #Get tables of polls
    polls = pd.read_html(r.text,header=0)[table_position]

    #Rename columns
    types = ["firm","date","error","size","lead","turnout"]
    new_cols = []
    for col in polls.columns:
        flag = 0
        for t in types:
            if t.lower() in col:
                new_cols.append(t)
                flag = 1
                break
        if flag == 0:
            if "Unnamed" in col:
                new_cols.append(columns.pop(0))
            else:
                new_cols.append(col)
    print(p_url)
    polls.columns = new_cols
    print(polls.columns)
    polls["date"] = polls["date"].str.split("–").str[-1]
    polls = polls.loc[polls["date"].apply(check_not_number)]
    
    polls.to_csv("data/individual_tables/{}/{}.csv".format(country,p_url.split("/")[-1]))
    
    #all_tables.append(polls)

mkdir: cannot create directory ‘data/individual_tables/Spain’: File exists
https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1982
Index(['firm', 'date', 'UCD', 'PSOE', 'PCE', 'AP–PDP', 'CiU', 'PNV', 'CDS',
       'error', 'size', 'Lead'],
      dtype='object')
https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1986
Index(['firm', 'date', 'PSOE', 'AP-PDP-PL', 'IU', 'CiU', 'CDS', 'PNV', 'PRD',
       'error', 'size', 'Lead'],
      dtype='object')
https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1989
Index(['firm', 'date', 'size', 'Turnout', 'PSOE', 'AP', 'CDS', 'CiU', 'IU',
       'PNV', 'HB', 'EE', 'PA', 'ERC', 'CG', '/wiki/Galician_Coalition', 'AIC',
       'PAR', 'PSG–EG', 'LV', '/wiki/Canarian_Independent_Groups', 'UV',
       '/wiki/Galician_Socialist_Party%E2%80%93Galician_Left', 'ARM', 'Lead'],
      dtype='object')
https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_electi

In [150]:
country = "Spain"

#combine tables
all_tables = []
for table in os.listdir("./data/individual_tables/{}".format(country)):
    all_tables.append(pd.read_csv("data/individual_tables/{}/{}".format(country,table),parse_dates=["date"],na_values=["–","?"]))
    print(all_tables[-1]["date"].dtype)

datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]


In [151]:
all_tables = pd.concat(all_tables)
all_tables.to_csv("./data/all_tables_Spain.csv",index=None)

In [155]:
def clean_data(x):
    if isinstance(x,float):
        return x
    if "w." in x:
        return np.NaN
    number = x.split("/")[0].split("?")[0].split("–")[0].split("[")[0].strip()
    if len(number) == 0:
        return np.NaN
    if "<" in number:
        return np.NaN
    try:
        return float(number)
    except:
        print(number)
        print(len(number))
        

In [157]:
#clean data
spain = pd.read_csv("data/all_tables_Spain.csv",parse_dates=["date"])
for col in spain.columns:
    if col.lower() in types:
        continue
    if "Unnamed" in col:
        continue
    spain[col] = spain[col].apply(clean_data).astype(float)
    
    
spain["PSOE"] = spain[["PSOE",'/wiki/Galician_Socialist_Party%E2%80%93Galician_Left','/wiki/PSM%E2%80%93Nationalist_Agreement',"PSG–EG","PSM"]].sum(1)
spain["BNG"] = spain[["BNG","BNG–Nós","/wiki/Galician_Nationalist_Bloc"]].sum(1)
spain["IU"] = spain[["IU","IU–LV","IU–UPeC"]].sum(1)
spain["PP"] = spain[["PP","AP","AP-PDP-PL","AP–PDP"]].sum(1)
spain["Podemos"] = spain[["Podemos","Isotipo Unidos Podemos.svg"]].sum(1)
#spain = spain.dropna(thresh=len(spain)/5,axis=1)
spain.dropna(subset=['/wiki/Galician_Nationalist_Bloc'])[["BNG","/wiki/Galician_Nationalist_Bloc"]]

spain.to_csv("./data/all_tables_Spain_cleaned.csv")

In [158]:
spain.dtypes

/wiki/Canarian_Independent_Groups                              float64
/wiki/Confederation_of_the_Greens                              float64
/wiki/Galician_Coalition                                       float64
/wiki/Galician_Nationalist_Bloc                                float64
/wiki/Galician_Socialist_Party%E2%80%93Galician_Left           float64
/wiki/PSM%E2%80%93Nationalist_Agreement                        float64
/wiki/Progressives_for_the_Balearic_Islands                    float64
/wiki/Unity_for_the_Isles                                      float64
AIC                                                            float64
AP                                                             float64
AP-PDP-PL                                                      float64
AP–PDP                                                         float64
ARM                                                            float64
Amaiur logo.svg                                                float64
Aralar

# Systematical

In [ ]:
#Opinion polling by country
https://en.wikipedia.org/wiki/Category:Opinion_polling_by_country